In [37]:
# Useful starting lines
#%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from preprocessing_helpers import *
from Basics import *
#%load_ext autoreload
#%autoreload 2

In [38]:
DATA_TRAIN_PATH = 'data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [39]:
# split test-train set
indx = np.arange(y.shape[0])
np.random.shuffle( indx )
ind_max = int(y.shape[0] * (0.8))

trainY , trainF = y[indx[:ind_max]] , tX[indx[:ind_max]]
testY , testF = y[indx[ind_max:]] , tX[indx[ind_max:]]

In [40]:
####### Preprocessing ######################################################################

In [41]:
# normalize
trainF , mean , std = standardize(trainF)
trainF = np.insert(trainF ,0 , np.ones(trainF.shape[0]),axis = 1) # add bias
testF = (testF - mean)/std 
testF = np.insert(testF ,0 , np.ones(testF.shape[0]),axis = 1) # add bias

In [28]:
# replace -999

In [29]:
# replace outliers

In [30]:
# dataset balencing

In [51]:
# feature selection expansion
trainF_sel = select_and_expand_f(trainF)
testF_sel = select_and_expand_f(testF)

IndexError: index 20 is out of bounds for axis 1 with size 19

In [52]:
###### Training/Testing ######################################################################

In [54]:
w_initial = np.random.rand(trainF.shape[1])

w,_ = least_squares_GD( trainY,trainF_sel ,w_initial,200,0.1)

get_accuracy(w , testY ,testF_sel )

NameError: name 'trainF_sel' is not defined